In [1]:
import requests

In [3]:
def get_article_summary(
  text,
):
  url = "http://15.165.211.100:8000/models/summarize"  # 또는 EC2 내부/외부 주소
  payload = {"article": text}

  try:
      response = requests.post(url, json=payload)
      response.raise_for_status()

      summary = response.json()["summary"]  # 또는 실제 리턴 필드에 따라 조정

      return summary

  except Exception as e:
      print(f"❌ 요약 요청 실패: {e}")
      return ""

def get_stock_list(text):
  # 🟡 토큰화 및 입력값 준비

  url = "http://15.165.211.100:8000/models/stock_list"
  payload = {"article": text}

  try:
      response = requests.post(url, json=payload)
      response.raise_for_status()
      return response.json()["stock_list"]  # 혹은 API 응답 구조에 따라 조정
  except Exception as e:
      print(f"❌ 종목명 추출 실패: {e}")
      return []

In [10]:
text = '나이스신용평가는 23일 삼성바이오로직스의 인적 분할이 신용도에 끼치는 영향은 제한적이라고 밝혔다.\n삼성바이오로직스는 전날 22일 인적 분할 계획을 공시했다. 바이오의약품 개발 및 상업화 사업을 분할해 신설법인(삼성에피스홀딩스)을 설립하고, 존속법인(삼성바이오로직스㈜은 바이오의약품 위탁개발생산(CDMO) 사업을 영위할 예정이다. 삼성에피스홀딩스는 삼성바이오에피스 및 바이오 관련 신사업을 영위하는 신설 자회사를 100% 지배하는 지주사로서 자회사 관리와 신규투자 등을 담당하게 된다.\n나신평은 분할 전 발행된 회사채의 신용도에 즉각적인 변화는 없을 것이라고 설명했다. 나신평은 "인적 분할의 경우 상법상 분할 전 회사의 채무에 관하여 각 분할 존속회사 및 분할 신설회사가 연대해 변제할 책임이 있다"며 "가상적인 연결 실체의 상환능력에는 변화가 없으며, 기존 회사채의 신용도는 동일하게 유지된다"고 설명했다.\n그러면서 인적 분할이 회사의 신용도에 미치는 영향은 제한적인 수준으로 판단된다"며 "분할 이후 회사의 매출 외형은 분할 전 대비 약 23% 축소될 것으로 보인다"면서도 "PPA 상각비 등이 신설법인에 귀속되면서 회사의 이익창출력은 우수한 수준을 유지할 것으로 전망된다"고 말했다.\n다만 분할 이후 회사의 확정된 재무제표 상 재무안정성 지표의 저하 수준과 매출, 수익성 및 현금창출력의 변동 여부와 함께 향후 투자계획의 변경 등이 신용도에 영향을 미칠 것으로 내다봤다. 나신평은 "회사의 재무상태는 최종 분할 시점에 변경될 가능성이 존재하며, 이후에도 분할대상 부문의 사업 조정 과정에서 추가적인 변동이 나타날 수 있다"며 "향후 임시주주총회 투표 결과를 비롯한 기업분할 관련 진행 상황 및 분할 과정에서의 재무 안정성의 변화 여부를 지속 점검해 신용등급에 반영할 예정"이라고 설명했다.'

In [11]:
summary = get_article_summary(text)
summary

'나이스신용평가는 23일 삼성바이오로직스의 인적 분할 계획이 발표되었으며 이에 따라 회사의 신용도에 미치는 영향은 제한적이라고 밝혔다.'

In [9]:
stock_list = get_stock_list(summary)
stock_list

['삼성전자', 'MBK파트너스']

In [12]:
import pandas as pd 

In [15]:
df_krx = pd.read_csv('../../automation/db/KRX_KOSPI_STOCK.csv', encoding='cp949')

In [16]:
df_krx["종목코드"] = df_krx["종목코드"].astype(str).str.zfill(6)

In [18]:
df_krx.drop(columns=['Unnamed: 0'], inplace=True)
df_krx.head()

,종목코드,종목명,종가,대비,등락률,상장시가총액
0,005930,삼성전자,55700,1800,3.34,329723832.0
1,000660,SK하이닉스,208000,5500,2.72,151424492.0
2,207940,삼성바이오로직스,1029000,3000,0.29,73238046.0
3,373220,LG에너지솔루션,282500,10000,3.67,66105000.0
4,012450,한화에어로스페이스,884000,-3000,-0.34,41809842.0


In [20]:
df_krx.to_csv('../../automation/db/KRX_KOSPI_STOCK.csv', index=False)

In [21]:
df_krx_desc = pd.read_csv("../../automation/db/KRX_KOSPI_DESCRIPTION.csv", encoding="cp949")

In [22]:
df_krx_desc["종목코드"] = df_krx_desc["종목코드"].astype(str).str.zfill(6)

In [24]:
df_krx_desc.drop(columns=["Unnamed: 0"], inplace=True)
df_krx_desc.head()

,종목코드,종목명,시장구분,소속부,소속부.1,업종코드,업종명,결산월,지정자문인,상장주식수,액면가,자본금,통화구분,대표이사,대표전화,주소
0,095570,AJ네트웍스,KOSPI,NaN,NaN,147603,산업용 기계 및 장비 임대업,12,NaN,45252759,1000,46822295000,원(KRW),곽정호,02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
1,006840,AK홀딩스,KOSPI,NaN,NaN,116409,기타 금융업,12,NaN,13247561,5000,66237805000,원(KRW),"고준, 채형석 (각자 대표이사)",02-768-2923,서울특별시 마포구 양화로 188 -
2,027410,BGF,KOSPI,NaN,NaN,116409,기타 금융업,12,NaN,95716791,1000,95716791000,NaN,홍정국,1577-3663,서울특별시 강남구 테헤란로 405
3,282330,BGF리테일,KOSPI,NaN,NaN,74701,종합 소매업,12,NaN,17283906,1000,17283906000,원(KRW),민승배,02-1577-8007,서울특별시 강남구 테헤란로 405 BGF사옥
4,138930,BNK금융지주,KOSPI,NaN,NaN,116409,기타 금융업,12,NaN,318383519,5000,1629676230000,원(KRW),빈대인,051-620-3023,부산광역시 남구 문현금융로 30(문현동)


In [25]:
df_krx.to_csv("../../automation/db/KRX_KOSPI_DESCRIPTION.csv", index=False)

In [ ]:
# ③ 종목명 → 종목코드 딕셔너리 생성
name_to_code = dict(zip(df_krx["종목명"], df_krx["종목코드"]))

# ④ stock_list를 매핑된 딕셔너리 리스트로 변환하는 함수
import ast


def enrich_stock_list(stock_names_raw):
    try:
        stock_names = stock_names_raw
        result = []
        for name in stock_names:
            code = name_to_code.get(name)
            if code:
                result.append({"stock_id": code, "stock_name": name})
        return result
    except Exception:
        return []


# ⑤ 적용
# df_meta["stock_list"] = df_meta["stock_list"].apply(enrich_stock_list)

In [39]:
stock_list2 = enrich_stock_list(stock_list)
stock_list2

[{'stock_id': '005930', 'stock_name': '삼성전자'}]